In [ ]:
%%capture
!pip install neptune-client psutil
!git clone https://github.com/Cho-D-YoungRae/URP_PD.git
%cd URP_PD
!pwd

In [ ]:
import dataset
import object_detection
from utils import *

import torch
from torch.utils.data import Dataset, DataLoader

import os
import json
import numpy as np
import argparse
from tqdm.auto import tqdm
import time
from datetime import datetime

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using <{device}> device")

Using <cuda> device


In [ ]:
# ====== constants ======#
label_map = {'background': 0, 'person': 1}
rev_label_map = {v: k for k, v in label_map.items()} 

## setting

In [ ]:
from torch.backends import cudnn
cudnn.benchmark = True

# ====== Random Seed Initialization ====== #
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.baselineID = 15

# ====== Dataset ====== #
args.val_split = 0.1
args.ch_option = {'num_ch': 3,
                  'img_type': 'visible'}

# ====== Model ====== #
args.base_model = 'VGG16bnBase'
args.n_classes = len(label_map)


# ====== Optimizer & Training ====== #
args.optim = 'Adam' 
args.lr = 1e-3
args.twice_b_lr = True
args.weight_decay = 5e-4

args.epochs = 100
args.train_batch_size = 32
args.test_batch_size = 64

args.decay_lr_at = [int(args.epochs/6)*4,
                    int(args.epochs/6)*5]
args.decay_lr_to = 0.1

## neptune init

In [ ]:
import neptune.new as neptune

api_token = 
run = neptune.init(
    project='jodyr/urp',
    source_files=['*.py'],
    api_token=api_token,
    # run='PD-25',
    )

run["parameters"] = vars(args)


https://app.neptune.ai/jodyr/urp/e/PD-45
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


## train

In [ ]:
def train(train_loader, model, criterion, optimizer):
    """
    One epoch's training.

    :param train_loader: DataLoader for training data
    :param model: model
    :param criterion: MultiBox loss
    :param optimizer: optimizer
    :param epoch: epoch number
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.train()  # training mode enables dropout

    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    print_freq = len(train_loader) // 4

    start = time.time()

    # Batches
    for i, (images, bboxes, category_ids, is_crowds) in enumerate(train_loader):
        data_time.update(time.time() - start)

        images = images.to(device)  # (batch_size (N), 3, 300, 300)
        bboxes = [b.to(device) for b in bboxes]
        category_ids = [c.to(device) for c in category_ids]

        # Forward prop.
        predicted_locs, predicted_scores = model(images)  # (N, 8732, 4), (N, 8732, n_classes)

        # Loss
        loss = criterion(predicted_locs, predicted_scores, bboxes, category_ids)  # scalar

        # Backward prop.
        optimizer.zero_grad()
        loss.backward()

        # Update model
        optimizer.step()

        losses.update(loss.item(), images.size(0))
        batch_time.update(time.time() - start)

        start = time.time()

        # Print status
        if i % print_freq == 0:
            print(f'[{i}/{len(train_loader)}]\t'
                  f'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  f'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  f'Loss {losses.val:.4f} ({losses.avg:.4f})\t')
    del predicted_locs, predicted_scores, images, bboxes, category_ids

    train_loss = losses.avg
    return train_loss        

## validation

In [ ]:
def validation(val_loader, model, criterion):
    model.eval()

    num_batches = len(val_loader)
    losses = AverageMeter()
    with torch.no_grad():
        for i, (images, bboxes, category_ids, _) in enumerate(val_loader):
            images = images.to(device)
            bboxes = [b.to(device) for b in bboxes]
            category_ids = [l.to(device) for l in category_ids]

            predicted_locs, predicted_scores = model(images)
            loss = criterion(predicted_locs, predicted_scores, bboxes, category_ids).item()

            losses.update(loss, images.size(0))

    val_loss = losses.avg
    return val_loss

## checkpoint

In [ ]:
checkpoint = os.path.join('/content/drive/MyDrive/2021.summer_URP/PD/checkpoint',
                          str(args.baselineID)+'.pth.tar')
checkpoint = checkpoint if os.path.isfile(checkpoint) else None
print(f"checkpoint: {checkpoint}")

checkpoint: None


In [ ]:
if checkpoint is None:
    start_epoch = 1
    lr = args.lr
    model = object_detection.SSD300(n_classes=args.n_classes,
                                    base=args.base_model,
                                    ch_option=args.ch_option)
    if args.twice_b_lr:
        biases = list()
        not_biases = list()
        for param_name, param in model.named_parameters():
            if param.requires_grad:
                if param_name.endswith('.bias'):
                    biases.append(param)
                else:
                    not_biases.append(param)
        optimizer = getattr(torch.optim, args.optim)(params=[{'params': biases, 'lr': 2 * lr}, 
                                                            {'params': not_biases}],
                                                     lr=lr,
                                                     weight_decay=args.weight_decay)
    else:
        optimizer = getattr(torch.optim, args.optim)(params=model.parameters(),
                                                     lr=lr,
                                                     weight_decay=args.weight_decay)

else:
    checkpoint = torch.load(checkpoint)
    start_epoch = checkpoint['epoch'] + 1
    print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
    model = checkpoint['model']
    optimizer = checkpoint['optimizer']


model = model.to(device)
criterion = object_detection.MultiBoxLoss(priors_cxcy=model.priors_cxcy).to(device)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth




Loaded base model.



/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


## dataset init

In [ ]:
workers = 4
train_dataset = dataset.KaistPDDataset(ch_option=args.ch_option)
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=args.train_batch_size, 
                                           shuffle=True,
                                           collate_fn=dataset.collate_fn,
                                           num_workers=workers,
                                           pin_memory=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## experiment

In [ ]:
checkpoint_dir = '/content/drive/MyDrive/2021.summer_URP/PD/checkpoint'
checkpoint_path = os.path.join(checkpoint_dir,
                               str(args.baselineID)+'.pth.tar')

In [ ]:
epochs = args.epochs
decay_lr_at = args.decay_lr_at
save_freq = 5


# Epochs
for epoch in range(start_epoch, epochs+1):
    print(f"# ====== Epoch {epoch} ====== # {datetime.now()}")
    # Decay learning rate at particular epochs
    if epoch in decay_lr_at:
        adjust_learning_rate(optimizer, args.decay_lr_to)

    # One epoch's training
    train_loss = train(train_loader=train_loader,
                       model=model,
                       criterion=criterion,
                       optimizer=optimizer)
    run['train/loss'].log(train_loss)
    if epoch % save_freq == 0:
        save_checkpoint(epoch, model, optimizer, checkpoint_path)

# ====== Epoch 76 ====== # 2021-07-29 01:28:37.378422


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[0/273]	Batch Time 104.415 (104.415)	Data Time 98.453 (98.453)	Loss 2.2782 (2.2782)	
[68/273]	Batch Time 14.054 (5.435)	Data Time 13.385 (4.765)	Loss 2.1408 (2.0304)	
[136/273]	Batch Time 13.859 (4.708)	Data Time 13.284 (4.074)	Loss 2.0535 (2.0600)	
[204/273]	Batch Time 14.384 (4.472)	Data Time 13.782 (3.849)	Loss 1.9682 (2.0585)	
[272/273]	Batch Time 12.094 (4.332)	Data Time 8.189 (3.706)	Loss 2.3579 (2.0606)	
# ====== Epoch 77 ====== # 2021-07-29 01:48:20.249004
[0/273]	Batch Time 11.222 (11.222)	Data Time 10.539 (10.539)	Loss 2.1969 (2.1969)	
[68/273]	Batch Time 8.118 (2.747)	Data Time 7.503 (2.079)	Loss 2.0442 (2.0700)	
[136/273]	Batch Time 6.554 (2.635)	Data Time 5.879 (1.966)	Loss 2.1740 (2.0569)	
[204/273]	Batch Time 7.616 (2.614)	Data Time 6.936 (1.945)	Loss 2.0764 (2.0534)	
[272/273]	Batch Time 0.648 (2.595)	Data Time 0.171 (1.927)	Loss 2.2267 (2.0560)	
# ====== Epoch 78 ====== # 2021-07-29 02:00:08.860641
[0/273]	Batch Time 10.765 (10.765)	Data Time 10.064 (10.064)	Loss 2.200

In [15]:
import eval

checkpoint = torch.load(checkpoint_path)
save_epoch = checkpoint['epoch']
print(f'Loaded checkpoint from epoch {save_epoch}\n')
model = checkpoint['model']
eval.evaluate(model, ch_option=args.ch_option)

Loaded checkpoint from epoch 100



/content/URP_PD/object_detection.py:393: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_boxes.append(class_decoded_locs[1 - suppress])
/content/URP_PD/object_detection.py:395: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_scores.append(class_scores[1 - suppress])



Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.35s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[55:10000000000] | visibility=[none+partial_occ] ] = 36.58%
Recall: 0.770941438102298


In [ ]:
import glob
import eval

checkpoints = f"/content/drive/MyDrive/2021.summer_URP/PD/checkpoints/{args.baselineID}/*.tar"
for checkpoint in glob.glob(checkpoints):
    checkpoint = torch.load(checkpoint)
    save_epoch = checkpoint['epoch']
    print(f'Loaded checkpoint from epoch {save_epoch}')
    model = checkpoint['model']
    eval.evaluate(model, ch_option=args.ch_option,
                  json_path=f'save_epoch_at_{save_epoch}')